<a href="https://colab.research.google.com/github/nwkrix/.cc/blob/Deep-Learning/prodClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
"""
    The aim of this deep learning model is to predict product class based on
    a custimer's complaint text message
    #####################################
    master is now named Deep-Learning
    If you have a local clone, you can update it by running the following commands.

    git branch -m master Deep-Learning
    git fetch origin
    git branch -u origin/Deep-Learning Deep-Learning
    git remote set-head origin -a
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

# test Tensorflow version
print("Tensorflow version # is ", tf.__version__)

Tensorflow version # is  2.14.0


In [62]:
df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/df8e341206e54c834aec/raw/48b1748ddf9a8e9e4516d2901973e72429f2e22c/Consumer_Complaints.csv', encoding = 'latin-1')
df.head()

<ipython-input-62-bcb83278d61c>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/df8e341206e54c834aec/raw/48b1748ddf9a8e9e4516d2901973e72429f2e22c/Consumer_Complaints.csv', encoding = 'latin-1')


,Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
0,1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219.0,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,NaN
1,1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006.0,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,NaN
2,1114255,Bank account or service,Checking account,Deposits and withdrawals,NaN,NY,11102.0,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, ...",In progress,Yes,NaN
3,1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721.0,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,NaN
4,1115890,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,FL,33461.0,Web,11/12/2014,11/13/2014,TransUnion,In progress,Yes,NaN


In [63]:
col = ['Issue','Product']
df = df[col]
df = df[pd.notnull(df['Issue'])]
df.head()

,Issue,Product
0,Disclosure verification of debt,Debt collection
1,Disclosure verification of debt,Debt collection
2,Deposits and withdrawals,Bank account or service
3,Communication tactics,Debt collection
4,Incorrect information on credit report,Credit reporting


In [64]:
df.isnull().sum() # verify that no entry is null

Issue      0
Product    0
dtype: int64

In [65]:
df['Product'].value_counts() # number of enteries of each product (type) in the data frame

Mortgage                   125752
Debt collection             44372
Credit card                 41563
Credit reporting            41214
Bank account or service     38071
Student loan                 9432
Consumer loan                9385
Payday loan                  1579
Money transfers              1540
Name: Product, dtype: int64

In [66]:
trainSize = int(len(df)*.8)
print('Train Size is %d' % trainSize)
print('Test Size is %d' % (len(df)-trainSize))

Train Size is 250326
Test Size is 62582


In [67]:
train_x = df['Issue'][:trainSize]
train_y = df['Product'][:trainSize]

test_x = df['Issue'][trainSize:]
test_y = df['Product'][trainSize:]

In [68]:
# Tokenize the independent variables, where # Tokenizer converts words data features into numbers
max_word = 1000
tokenize = text.Tokenizer(num_words=max_word, char_level=False)
tokenize.fit_on_texts(train_x)
x_train = tokenize.texts_to_matrix(train_x)
x_test = tokenize.texts_to_matrix(test_x)

In [69]:
# Encode the dependent variables. We can do that using sklearn utility to convert label strings to numbers
encoder = LabelEncoder()
encoder.fit(train_y)
y_train = encoder.transform(train_y)
y_test = encoder.transform(test_y)
# Next, convert to one-hot encoding rep
maxVal = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, maxVal)
y_test = utils.to_categorical(y_test,maxVal)
# Let's do a little bit of inspection
print('x_train shape ', (x_train.shape))
print('x_test shape ', (x_test.shape))
print('y_train shape ', (y_train.shape))
print('y_test shape ', (y_test.shape))

x_train shape  (250326, 1000)
x_test shape  (62582, 1000)
y_train shape  (250326, 9)
y_test shape  (62582, 9)


In [70]:
# Creating a neural network model
model = Sequential()
model.add(Dense(512,input_shape=(max_word,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(maxVal))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # the adam optimizer for large dataset

In [71]:
# Train the model
hist = model.fit(x_train, y_train, batch_size = 32, epochs = 1, verbose = 1, validation_split = 0.1)

7041/7041 [==============================] - 69s 10ms/step - loss: 0.0319 - accuracy: 0.9883 - val_loss: 0.0316 - val_accuracy: 0.9804


In [72]:
# Evaluate the model accuracy
score = model.evaluate(x_test, y_test,batch_size = 32, verbose = 1)
print('Test score ', score[0])
print('Test accuracy ', score[1])

1956/1956 [==============================] - 7s 4ms/step - loss: 0.0477 - accuracy: 0.9684
Test score  0.04769957438111305
Test accuracy  0.9683934450149536


In [73]:
#Test some prediction
txt_labels = encoder.classes_
for i in range(100):
    predictn = model.predict(np.array([x_test[i]]))
    p_label = txt_labels[np.argmax(predictn)]
    print(test_x.iloc[i][:50], '...')
    print('Actual label is> '+ test_y.iloc[i])
    print('Predicted label is> '+ p_label + '\n')

1/1 [==============================] - 0s 53ms/step
Loan servicing, payments, escrow account ...
Actual label is> Mortgage
Predicted label is> Mortgage

1/1 [==============================] - 0s 20ms/step
Loan modification,collection,foreclosure ...
Actual label is> Mortgage
Predicted label is> Mortgage

1/1 [==============================] - 0s 24ms/step
Deposits and withdrawals ...
Actual label is> Bank account or service
Predicted label is> Bank account or service

1/1 [==============================] - 0s 19ms/step
Credit line increase/decrease ...
Actual label is> Credit card
Predicted label is> Credit card

1/1 [==============================] - 0s 23ms/step
Managing the loan or lease ...
Actual label is> Consumer loan
Predicted label is> Consumer loan

1/1 [==============================] - 0s 20ms/step
Identity theft / Fraud / Embezzlement ...
Actual label is> Credit card
Predicted label is> Credit card

1/1 [==============================] - 0s 21ms/step
Account opening, closi